# Programmering, gör din egna självinlärda artificiella intelligens

In [1]:
# BlackJack - By Rickard Maus

import random
import numpy as np

deck = {"ace1" : 1, "ace2" : 1, "ace3" : 1, "ace4" : 1, "two1" : 2, "two2" : 2, "two3" : 2, "two4" : 2,
       "thr1" : 3, "thr2" : 3, "thr3" : 3, "thr4" : 3, "for1" : 4, "for2" : 4, "for3" : 4, "for4" : 4,
       "fiv1" : 5, "fiv2" : 5, "fiv3" : 5, "fiv4" : 5, "six1" : 6, "six2" : 6, "six3" : 6, "six4" : 6,
       "sev1" : 7, "sev2" : 7, "sev3" : 7, "sev4" : 7, "eit1" : 8, "eit2" : 8, "eit3" : 8, "eit4" : 8,
       "nin1" : 9, "nin2" : 9, "nin3" : 9, "nin4" : 9, "ten1" : 10, "ten2" : 10, "ten3" : 10, "ten4" : 10,
       "J1" : 10, "J2" : 10, "J3" : 10, "J4" : 10, "Q1" : 10, "Q2" : 10, "Q3" : 10, "Q4" : 10,
       "K1" : 10, "K2" : 10, "K3" : 10, "K4" : 10}

class b_jack():
    def __init__(self):
        self.deck_L = list(deck)
        self.Player_hand = []
        self.Dealer_hand = []
        self.hold = np.array([])
        self.Player_hand_size = 0
        self.Dealer_hand_size = 0
        self.reward = 0
        self.pull_c = 2
        self.n_Player = 2
        self.bustvar = False
        self.tie = False
        self.arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(1,12)
        
    def arr_start_assign(self):
        self.arr[0,0] = deck[self.Player_hand[0]]
        self.arr[0,1] = deck[self.Player_hand[1]]
        
    def player_hand_count(self):
        P_hand_s = len(self.Player_hand)
        self.Player_hand_size = 0
        for n in range(0, P_hand_s):
            self.Player_hand_size = self.Player_hand_size + deck[self.Player_hand[n]]
    
    def dealer_hand_count(self):
        D_hand_s = len(self.Dealer_hand)
        self.Dealer_hand_size = 0
        for n in range(0, D_hand_s):
            self.Dealer_hand_size = self.Dealer_hand_size + deck[self.Dealer_hand[n]]
        
    def bust(self, x):
        Bust = 0
        hand_s = len(x)
        for n in range(0, hand_s):
            Bust = Bust + deck[x[n]]
        if (Bust > 21) and (x == self.Player_hand):
            self.reward = -1
        elif (Bust > 21) and (x == self.Dealer_hand):
            self.bustvar = True
            self.reward = 1
    
    def deal(self, x):
        draw = ''.join(random.sample(self.deck_L,1))
        self.deck_L.remove(draw)
        if x%2 == 1:
            self.Player_hand.append(draw)
        else:
            self.Dealer_hand.append(draw)
            
    def game_start(self):
        for x in range(0,4):
            self.deal(x)
        self.arr_start_assign()
        self.player_hand_count()

    def mid_game(self, x):
        if x == 0:
            self.reward = 0
            self.deal(1)
            self.player_hand_count()
            self.bust(self.Player_hand)
            self.arr[0,self.pull_c] = deck[self.Player_hand[self.n_Player]]
            self.pull_c += 1
            self.n_Player += 1
        else:
            self.end_game()
        return self.arr, self.reward
            
    def end_game(self):
        for x in range(0,12):
            self.dealer_hand_count()
            self.player_hand_count()
            self.bust(self.Dealer_hand)
            if (self.bustvar == True):
                break
            elif (self.Dealer_hand_size > self.Player_hand_size):
                self.reward = -1
                break
            elif (self.Dealer_hand_size == self.Player_hand_size):
                self.tie = True
                self.reward = -1
                break
            elif (self.Dealer_hand_size < 17):
                self.deal(0)
            else:
                self.reward = 1
                break
            

In [2]:
import tensorflow as tf

def NALU(prev_layer, num_outputs):
    eps=1e-7
    shape = (int(prev_layer.shape[-1]),num_outputs)

    # NAC cell
    W_hat = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
    M_hat = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
    W = tf.tanh(W_hat) * tf.sigmoid(M_hat)
    a = tf.matmul(prev_layer, W)
    G = tf.Variable(tf.truncated_normal(shape, stddev=0.02))
    
    # NALU
    m = tf.exp(tf.matmul(tf.log(tf.abs(prev_layer) + eps), W))
    g = tf.sigmoid(tf.matmul(prev_layer, G))
    out = g * a + (1 - g) * m
    return out

In [3]:
import tensorflow as tf
import time
learning_rate=0.0000007

graph = tf.Graph()
with graph.as_default():
    tf_hand = tf.placeholder(tf.float32, shape=(1,12))
    nextQ = tf.placeholder(tf.float32, shape=[1,2])
    
    X_1 = NALU(tf_hand,12)
    Y_layer_out = NALU(X_1,2)
    
    '''
    weights = {
        'W1' : tf.Variable(tf.random_uniform([12,12],0,1))
        ,'W2' : tf.Variable(tf.random_uniform([12,2],0,1))
    }
    '''
    
    '''
    def Q_model(x):
        layer_1 = tf.matmul(x, weights['W1'])
        layer_out = tf.matmul(tf.nn.relu(layer_1), weights['W2'])
        return layer_out
    '''

    loss = tf.reduce_sum(tf.square(nextQ - Y_layer_out))
    trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    updateModel = trainer.minimize(loss)
    
    Q_values = Y_layer_out
    predict = tf.argmax(Q_values,1)
    

In [ ]:
y = 0.9
num_episodes = 1000000
e = 0.5
e_lower_limit = 0.01
annealing = (e-e_lower_limit)/num_episodes
actions = [0, 1]
tot_win = 0
win = 0
lose = 0
losstotavrg = 0
winrate_list = []
loserate_list = []
Loss_list = []
list_draw_statistics = []
draw_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
point_total_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
miniepoch = 25000

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for i in range(num_episodes):
        game = b_jack()
        game.game_start()
        s = game.arr
        hey = False
        Losstot = 0
        if (i % miniepoch == 0):
            print(s)
        for j in range(1,13):
            prev = None
            game.hold,allQ = sess.run([predict,Q_values], feed_dict={tf_hand:s})
            if np.random.rand(1) < e:
                prev = game.hold
                game.hold[0] = random.sample(actions,1)[0]
            point_total_statistics[game.Player_hand_size-2] += 1
            if game.hold[0] == 0:
                draw_statistics[game.Player_hand_size-2] += 1
            s1,r = game.mid_game(game.hold[0])
            if (i % miniepoch == 0):
                print(allQ)
                print(game.hold, prev)
                print(game.Dealer_hand, "Dealer hand")
                print(r, " r")
                print(s1)
                hey = True
                print("Winrate: {}%".format(100*win/miniepoch))
            Q1 = sess.run(Q_values,feed_dict={tf_hand:s1})
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,game.hold[0]] = r + y*maxQ1
            _,L = sess.run([updateModel,loss],feed_dict={tf_hand:s,nextQ:targetQ})
            Losstot += L
            s = s1
            if (r == 1) or game.tie:
                e = e-annealing
                if r == 1:
                    win += 1
                    tot_win += 1
                break
            elif (r == -1):
                lose += 1
                break
        losstotavrg += (Losstot/j)
        if hey == True and (i != 0):
            for n in range(0,20):
                draw_statistics[n] = (draw_statistics[n]/point_total_statistics[n])*100
            print(draw_statistics)
            print(e)
            list_draw_statistics.append(draw_statistics)
            draw_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            point_total_statistics = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            Loss_list.append((losstotavrg/miniepoch))
            winrate_list.append((win/miniepoch)*100)
            loserate_list.append((lose/miniepoch)*100)
            losstotavrg = 0
            win = 0
            lose = 0
            print("---------------------")
        elif hey == True:
            print("---------------------")
print(Loss_list)
print(winrate_list)
print(loserate_list)
print("Wins:", win)
print("Losses", lose)
print("Winrate after {} games lr={}: {}%".format(num_episodes, learning_rate, 100*win_tot/num_episodes))
            

[[ 3 10  0  0  0  0  0  0  0  0  0  0]]
[[0.49181098 0.5016828 ]]
[1] None
['ten4', 'sev4'] Dealer hand
-1  r
[[ 3 10  0  0  0  0  0  0  0  0  0  0]]
Winrate: 0.0%
---------------------
[[ 3 10  0  0  0  0  0  0  0  0  0  0]]
[[0.37649027 0.37613636]]
[0] None
['Q1', 'six1'] Dealer hand
0  r
[[ 3 10  5  0  0  0  0  0  0  0  0  0]]
Winrate: 15.056%
[[0.3880669  0.39069557]]
[1] None
['Q1', 'six1', 'K3'] Dealer hand
1  r
[[ 3 10  5  0  0  0  0  0  0  0  0  0]]
Winrate: 15.056%
[58.59375, 64.25992779783394, 65.77777777777779, 65.4434250764526, 64.29512516469038, 61.73212487411883, 58.77049180327869, 60.37735849056604, 54.438642297650134, 51.310997815003645, 52.908891328210764, 51.85185185185185, 50.697849867974355, 47.38848646587876, 51.34387351778656, 50.89038385437278, 54.80649188514357, 55.63439065108514, 60.04585841215248, 61.2272420768712]
0.4972285599998889
---------------------
[[4 6 0 0 0 0 0 0 0 0 0 0]]
[[0.0591832 0.0583497]]
[0] None
['Q2', 'eit4'] Dealer hand
0  r
[[4 6 1 0 0 

[[ 4 10  0  0  0  0  0  0  0  0  0  0]]
[[-2.7347376 -2.826552 ]]
[0] None
['thr1', 'six1'] Dealer hand
-1  r
[[ 4 10  8  0  0  0  0  0  0  0  0  0]]
Winrate: 26.76%
[73.55371900826447, 77.47440273037543, 72.99270072992701, 76.97262479871175, 77.03703703703704, 78.09045226130652, 78.29787234042553, 76.53478854024557, 75.45507927187316, 76.4280798348245, 76.80851063829788, 75.64363143631437, 75.5315558555518, 72.22407469156386, 54.207232267037554, 51.031772027513924, 33.252678880055306, 20.615275813295618, 22.04663212435233, 23.352272727272727]
0.447411729997892
---------------------
[[2 5 0 0 0 0 0 0 0 0 0 0]]
[[-2.2910674 -2.6031554]]
[0] None
['K3', 'thr1'] Dealer hand
0  r
[[ 2  5 10  0  0  0  0  0  0  0  0  0]]
Winrate: 25.708%
[[-2.599065 -2.583125]]
[0] [0]
['K3', 'thr1'] Dealer hand
0  r
[[ 2  5 10  1  0  0  0  0  0  0  0  0]]
Winrate: 25.708%
[[-2.6474564 -2.4463732]]
[1] [1]
['K3', 'thr1', 'K4'] Dealer hand
1  r
[[ 2  5 10  1  0  0  0  0  0  0  0  0]]
Winrate: 25.708%
[83.4782